In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# R  W  11111111
# R  P  1111111
# G  W  1111111
# G  P  1111111
# H  W  1111111
# H  P  1111111

category = 'R'
# category = 'G'
# category = 'H'

bet_type = 'W'
# bet_type = 'P'

In [3]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfoo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfoo)))

21699 races loaded!


In [4]:
dfo = dfoo.loc[dfoo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

7057 R races!


In [5]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jaco/.pyenv/versions/3.6.1/envs/tabby/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,quinella,exacta,trifecta,first_four,results,runners
1025,1610,RANDWICK,NSW,S,R,2017-09-16,1,TAB HIGHWAY HANDICAP,2017-09-16 02:15:00,Paying,1000,"[[4], [10], [2], [5]]",11,"[{""runnerName"": ""AZZIE'S READY"", ""runnerNumber...",605.4,NaN,NaN,NaN,"[[4], [10], [2], [5]]","[{'runnerName': 'AZZIE'S READY', 'runnerNumber..."
1026,1611,RANDWICK,NSW,S,R,2017-09-16,2,SHOOT OUT MILE,2017-09-16 02:50:00,Paying,1600,"[[3], [5], [8], [7]]",10,"[{""runnerName"": ""GOLD HORIZON"", ""runnerNumber""...",2296.9,NaN,NaN,NaN,"[[3], [5], [8], [7]]","[{'runnerName': 'GOLD HORIZON', 'runnerNumber'..."
1027,1612,RANDWICK,NSW,S,R,2017-09-16,3,SCHWEPPES HERITAGE STAKES,2017-09-16 03:25:00,Paying,1100,"[[2], [1], [8], [6]]",10,"[{""runnerName"": ""LUCKY LOUIE"", ""runnerNumber"":...",1421.1,NaN,NaN,NaN,"[[2], [1], [8], [6]]","[{'runnerName': 'LUCKY LOUIE', 'runnerNumber':..."
1028,1613,RANDWICK,NSW,S,R,2017-09-16,4,BILL RITCHIE HANDICAP,2017-09-16 04:00:00,Paying,1400,"[[7], [3], [4], [6]]",9,"[{""runnerName"": ""PAJARO"", ""runnerNumber"": 6, ""...",111.9,NaN,NaN,NaN,"[[7], [3], [4], [6]]","[{'runnerName': 'PAJARO', 'runnerNumber': 6, '..."
1029,1614,RANDWICK,NSW,S,R,2017-09-16,5,MCGRATH ESTATE TEA ROSE STAKES,2017-09-16 04:35:00,Paying,1400,"[[6], [2], [5], [1]]",12,"[{""runnerName"": ""PANDEMONIUM"", ""runnerNumber"":...",911.7,NaN,NaN,NaN,"[[6], [2], [5], [1]]","[{'runnerName': 'PANDEMONIUM', 'runnerNumber':..."


In [6]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['win_odds']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
#         runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,fixedOdds,has_odds,num_runners,...,place_rank,place_scaled,riderDriverName,runnerName,runnerNumber,trainerName,win_odds,win_perc,win_rank,win_scaled
3,0.024693,0.007582,4.544170e-03,4.409936e-03,1,1.5,0,"{'returnWin': 17, 'returnWinOpen': 17, 'return...",True,12,...,12.000000,0.039773,C BENNETT,RAVEN BANNER,9,DANIEL MARK LOUGHNANE,17.0,0.058824,11.000000,0.031574
4,0.017687,0.005431,3.133167e-37,3.040614e-37,5,-1.0,0,"{'returnWin': 10, 'returnWinOpen': 13, 'return...",True,12,...,10.000000,0.064360,D TUDHOPE,INVICTUS,10,DAVID LOUGHNANE,10.0,0.100000,7.000000,0.053675
5,0.154362,0.047399,3.301714e-02,3.204182e-02,7,-1.0,0,"{'returnWin': 18, 'returnWinOpen': 34, 'return...",True,12,...,11.000000,0.052833,T EAVES,STREET POET,11,M HERRINGTON,18.0,0.055556,12.000000,0.029820
6,0.043421,0.013333,0.000000e+00,0.000000e+00,8,-1.0,0,"{'returnWin': 8.5, 'returnWinOpen': 9, 'return...",True,12,...,9.000000,0.065552,G DOWNING,SIR JACK,12,A W CARROLL,8.5,0.117647,6.000000,0.063147
7,0.472000,0.144932,1.663021e-01,1.613896e-01,4,-1.0,0,"{'returnWin': 4.4, 'returnWinOpen': 6, 'return...",True,12,...,2.000000,0.117994,R KINGSCOTE,LA CELEBS VILLE,1,TOM DASCOMBE,4.4,0.227273,2.000000,0.121989
8,0.340951,0.104693,1.064044e-01,1.032613e-01,6,-1.0,0,"{'returnWin': 6, 'returnWinOpen': 9.5, 'return...",True,12,...,5.000000,0.090764,J FANNING,WARRIOR PRINCE,2,E A L DUNLOP,6.0,0.166667,5.000000,0.089459
9,0.236790,0.072709,3.627979e-02,3.520810e-02,5,3.0,3,"{'returnWin': 11, 'returnWinOpen': 10, 'return...",True,12,...,6.000000,0.072241,ROSSA RYAN,SAXO JACK,3,MRS S LEECH,11.0,0.090909,8.000000,0.048796
10,0.376015,0.115459,1.271012e-01,1.233467e-01,9,-1.0,0,"{'returnWin': 5, 'returnWinOpen': 5, 'returnPl...",True,12,...,4.000000,0.093153,T EAVES,AUSPICION,4,T P TATE,5.0,0.200000,4.000000,0.107351
11,0.619556,0.190241,3.082605e-01,2.991545e-01,6,2.5,1,"{'returnWin': 2.2, 'returnWinOpen': 5, 'return...",True,12,...,1.000000,0.147492,C RODRIGUEZ,CORNBOROUGH,5,MARK WALFORD,2.2,0.454545,1.000000,0.243979
12,0.000000,0.000000,0.000000e+00,0.000000e+00,2,-1.0,0,"{'returnWin': 0, 'returnWinOpen': 0, 'returnPl...",False,12,...,2.564949,0.000000,A BESCHIZZA,CRAFTSMANSHIP,6,ROBERT EDDERY,0.0,0.000000,2.564949,0.000000


In [7]:
# drop scratched
data = data_all.dropna(subset=['win_odds'])
data = data[(data.win_odds > 0)]
data.describe()

,P_pred,P_prob,W_pred,W_prob,barrierNumber,claimAmount,finishingPosition,num_runners,place_odds,place_perc,place_rank,place_scaled,runnerNumber,win_odds,win_perc,win_rank,win_scaled
count,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000,73495.000000
mean,0.262036,0.096020,0.093481,0.096020,5.954473,-0.347935,0.817784,11.395129,9.166524,0.346897,6.011582,0.096020,6.449758,21.399544,0.126420,6.039883,0.096020
std,0.189642,0.073947,0.102386,0.102483,3.633440,1.285772,1.281154,3.327273,98.032604,0.207230,3.718223,0.062505,3.960870,29.163680,0.130779,3.717350,0.093892
min,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,4.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.010000,0.000000,1.000000,0.000000
25%,0.116105,0.041363,0.024344,0.025207,3.000000,-1.000000,0.000000,9.000000,2.050000,0.181818,3.000000,0.047749,3.000000,6.000000,0.038462,3.000000,0.031128
50%,0.222102,0.079226,0.059948,0.062306,6.000000,-1.000000,0.000000,11.000000,3.150000,0.312500,6.000000,0.083598,6.000000,11.000000,0.083333,6.000000,0.065311
75%,0.374328,0.135163,0.128326,0.134141,8.000000,0.000000,2.000000,13.000000,5.440000,0.487805,8.000000,0.133239,9.000000,23.000000,0.166667,8.000000,0.130510
max,1.000000,0.920081,1.000000,0.999068,24.000000,5.000000,4.000000,24.000000,2250.000000,0.990099,23.000000,0.441762,24.000000,900.000000,0.990099,23.000000,0.820518


In [8]:
# get label data
Y = data['finishingPosition']

if bet_type == 'W':
    Y = (Y == 1)
elif bet_type == 'P':
    Y = (Y == 1) | (Y == 2) | ((Y == 3) & (data['num_runners'] >= 8))

#print(data['finishingPosition'].head(10))
Y = Y.astype(int)
# Y.head(10)
Y.describe()

count    73495.000000
mean         0.096211
std          0.294882
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: finishingPosition, dtype: float64

In [9]:
# xp = fixed odds perc
xp = data['win_perc']
xp.describe()

count    73495.000000
mean         0.126420
std          0.130779
min          0.000000
25%          0.038462
50%          0.083333
75%          0.166667
max          0.990099
Name: win_perc, dtype: float64

In [10]:
# xs = odds scaled
xs = data['win_scaled']
xs.describe()

count    73495.000000
mean         0.096020
std          0.093892
min          0.000000
25%          0.031128
50%          0.065311
75%          0.130510
max          0.820518
Name: win_scaled, dtype: float64

In [11]:
# xr = win rank
xr = data['win_rank']
xr.describe()

count    73495.000000
mean         6.039883
std          3.717350
min          1.000000
25%          3.000000
50%          6.000000
75%          8.000000
max         23.000000
Name: win_rank, dtype: float64

In [12]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

count    73495.000000
mean        11.395129
std          3.327273
min          4.000000
25%          9.000000
50%         11.000000
75%         13.000000
max         24.000000
Name: num_runners, dtype: float64

In [13]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

,win_perc,win_scaled,win_rank,num_runners
count,73495.000000,73495.000000,73495.000000,73495.000000
mean,0.126420,0.096020,6.039883,11.395129
std,0.130779,0.093892,3.717350,3.327273
min,0.000000,0.000000,1.000000,4.000000
25%,0.038462,0.031128,3.000000,9.000000
50%,0.083333,0.065311,6.000000,11.000000
75%,0.166667,0.130510,8.000000,13.000000
max,0.990099,0.820518,23.000000,24.000000


In [17]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 250
print('epochs = {}'.format(epochs))

tag = '{}30x30{}'.format(category, bet_type)
print('tag = {}'.format(tag))
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

input dimension = 4
epochs = 500
tag = R30x30W



In [18]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)

32416/|/[loss: 0.270, acc: 0.905]  55%|| 32416/58796 [00:18<00:01, 14527.67it/s]